<a href="https://colab.research.google.com/github/PuneethReddyM/FakeCurrencyDetectionSystem/blob/master/Fake_Currency_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/PuneethReddyM/FakeCurrencyDetectionSystem.git

Cloning into 'FakeCurrencyDetectionSystem'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 66 (delta 6), reused 47 (delta 0), pack-reused 3
Receiving objects: 100% (66/66), 6.42 MiB | 33.04 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [4]:
# Import all necessary modules
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model, load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt


In [5]:
#define height and width of the image
height=300
width=300

#create a ResNet50 model instance without the top layer as we will add our own top layer
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(height,width,3))

94765736/94765736 [==============================] - 1s 0us/step


In [6]:
#define directory containing training and validation data
train_dir="/content/FakeCurrencyDetectionSystem/Dataset/Training"
validation_dir="/content/FakeCurrencyDetectionSystem/Dataset/Validation"

#number of batches the data has to be divided into
batch_size=8

#create datagen and generator to load the data from training directory
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(height,width),batch_size=batch_size)

#create datagen and generator to load the data from validation directory
validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
validation_generator=validation_datagen.flow_from_directory(validation_dir,target_size=(height,width),batch_size=batch_size)


Found 35 images belonging to 2 classes.
Found 7 images belonging to 2 classes.


In [7]:
#our own model which will be added onto the ResNet50 model
def build_finetune_model(base_model,dropout,fc_layers,num_classes):
    for layer in base_model.layers:
        layer.trainable=False

    x=base_model.output
    x=Flatten()(x)
    for fc in fc_layers:
        x=Dense(fc,activation='relu')(x)
        x=Dropout(dropout)(x)

    predictions=Dense(num_classes,activation='softmax')(x)

    finetune_model=Model(inputs=base_model.input,outputs=predictions)

    return finetune_model

class_list=['Real','Fake'] #the labels of our data
FC_Layers=[1024,1024]
dropout=0.5

finetune_model=build_finetune_model(base_model,dropout=dropout,fc_layers=FC_Layers,num_classes=len(class_list))


In [8]:
#define number of epochs(the number of times the model will be trained) and number of training images
num_epochs=100
num_train_images=35

In [ ]:
#checkpoint in case anything goes wrong
checkpoint=ModelCheckpoint("Final_model.h5",monitor='val_acc',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',period=1)
early=EarlyStopping(monitor='val_acc',min_delta=0,patience=40,verbose=1,mode="auto")

#compile the model before using
finetune_model.compile(loss="categorical_crossentropy",optimizer=optimizers.SGD(lr=0.000001,momentum=0.9),metrics=['accuracy'])

#train the model
finetune_model.fit_generator(generator=train_generator,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=validation_generator,validation_steps=1,callbacks=[checkpoint,early])

#save the model
finetune_model.save_weights("Final_model.h5")

<ipython-input-9-c4ac6982538f>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  finetune_model.fit_generator(generator=train_generator,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=validation_generator,validation_steps=1,callbacks=[checkpoint,early])


Epoch 1/100
4/4 [==============================] - ETA: 0s - loss: 24338.4961 - accuracy: 0.4444

4/4 [==============================] - 24s 5s/step - loss: 24338.4961 - accuracy: 0.4444 - val_loss: 1203903.3750 - val_accuracy: 0.7143
Epoch 2/100
4/4 [==============================] - ETA: 0s - loss: 272779804672.0000 - accuracy: 0.4815

4/4 [==============================] - 21s 5s/step - loss: 272779804672.0000 - accuracy: 0.4815 - val_loss: 115927932064329891840.0000 - val_accuracy: 0.7143
Epoch 3/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.5625                        

4/4 [==============================] - 23s 6s/step - loss: nan - accuracy: 0.5625 - val_loss: nan - val_accuracy: 0.2857
Epoch 4/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4444

...

4/4 [==============================] - 23s 5s/step - loss: nan - accuracy: 0.4444 - val_loss: nan - val_accuracy: 0.2857
Epoch 18/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4062

4/4 [==============================] - 25s 6s/step - loss: nan - accuracy: 0.4062 - val_loss: nan - val_accuracy: 0.2857
Epoch 19/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4074

4/4 [==============================] - 20s 5s/step - loss: nan - accuracy: 0.4074 - val_loss: nan - val_accuracy: 0.2857
Epoch 20/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4375

4/4 [==============================] - 24s 7s/step - loss: nan - accuracy: 0.4375 - val_loss: nan - val_accuracy: 0.2857
Epoch 21/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4444

4/4 [==============================] - 19s 5s/step - loss: nan - accuracy: 0.4444 - val_loss: nan - val_accuracy: 0.2857
Epoch 22/100


In [ ]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'finetune_model' is already defined and loaded with weights
finetune_model.load_weights("Final_model.h5")

def test_fake_image(image_path):
    img = image.load_img(image_path, target_size=(300, 300))
    img = np.asarray(img)
    plt.imshow(img)
    img = np.expand_dims(img, axis=0)
    output = finetune_model.predict(img)
    if output[0][0] > output[0][1]:
        print("Prediction: Real")
    else:
        print("Prediction: Fake")


# Testing fake images
test_fake_image("/content/FakeCurrencyDetectionSystem/Dataset/Testing/Fake.jpeg")



In [ ]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'finetune_model' is already defined and loaded with weights
finetune_model.load_weights("Final_model.h5")


def test_real_image(image_path):
    img = image.load_img(image_path, target_size=(300, 300))
    img = np.asarray(img)
    plt.imshow(img)
    img = np.expand_dims(img, axis=0)
    output = finetune_model.predict(img)
    if output[0][0] > output[0][1]:
        print("Prediction: Fake")
    else:
        print("Prediction: Real")

# Testing real images
test_real_image("/content/FakeCurrencyDetectionSystem/Dataset/Testing/Real.jpg")
